##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pose Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect pose landmarks from images.

## Preparation

Let's start with installing MediaPipe.


In [3]:
!pip install -q mediapipe==0.10.0

Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models) for more information about this model bundle.

In [4]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

## Visualization utilities

In [9]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## Download test image

To demonstrate the Pose Landmarker API, you can download a sample image using the follow code. The image is from [Pixabay](https://pixabay.com/photos/girl-woman-fitness-beautiful-smile-4051811/).

In [30]:
!wget -q -O image.jpg https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg

In [31]:
import cv2
img = cv2.imread("image.jpg")
cv2.imshow("someImage", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [32]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## Running inference and visualizing the results

The final step is to run pose landmark detection on your selected image. This involves creating your PoseLandmarker object, loading your image, running detection, and finally, the optional step of displaying the image with visualizations.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python) to learn more about configuration options that this solution supports.


In [33]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.jpg")

# STEP 4: Detect pose landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("annotaded", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


Visualize the pose segmentation mask.

In [34]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2.imshow("mask", visualized_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [35]:

segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
img8 = cv2.convertScaleAbs(segmentation_mask, alpha=1.00)
### inflate mask
kernel = np.ones((25,25),np.uint8)
img8_inflated = cv2.dilate(img8, kernel, iterations=10)
masked = cv2.bitwise_and(annotated_image, annotated_image, mask=img8_inflated)
### for some reason, some cv2 operations flip the colors, so we flip the back
masked  = masked[:, :, ::-1]
cv2.imshow("resultMasked", masked)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Mask the hands only

In [36]:
import numpy
import cv2

contours = [numpy.array([[1,1],[10,50],[50,50]], dtype=numpy.int32) , numpy.array([[99,99],[99,60],[60,99]], dtype=numpy.int32)]

drawing = numpy.zeros([100, 100],numpy.uint8)
for cnt in contours:
    cv2.drawContours(drawing,[cnt],0,(255,255,255),2)

cv2.imshow('output',drawing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates

#### from: https://github.com/google/mediapipe/blob/master/docs/solutions/pose.md
###        https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67
###
### the left hand wrist index is: 15
### the right hand wrist index is: 16
img_h, img_w, cannel = annotated_image.shape
pose_landmarks_list = detection_result.pose_landmarks
# Loop through the detected poses to visualize.
for idx in range(len(pose_landmarks_list)):
  pose_landmarks = pose_landmarks_list[idx]
  ### do rect around all landmarks
  coords = [_normalized_to_pixel_coordinates(l.x, l.y, img_w, img_h) for l in pose_landmarks]
  rect = cv2.minAreaRect(np.array(coords))
  box = cv2.boxPoints(rect)
  box = np.intp(box)
  boxed_annotated = cv2.drawContours(annotated_image.copy(),[box],0,(0,0,255),2)
  ### do rect around both hands
  w = 200
  h = 300
  wh = w / 2
  hh = h / 2
  handcenterLx, handcenterLy = coords[15]
  p1 = (int(handcenterLx-wh), int(handcenterLy-hh))
  p2 = (int(handcenterLx+wh), int(handcenterLy+hh))
  cv2.rectangle(boxed_annotated, p1, p2, (255,0,0), 2)
  handcenterRx, handcenterRy = coords[16]
  p1 = (int(handcenterRx-wh), int(handcenterRy-hh))
  p2 = (int(handcenterRx+wh), int(handcenterRy+hh))
  cv2.rectangle(boxed_annotated, p1, p2, (255,0,0), 2)

cv2.imshow("hand_bb_%s" % idx, boxed_annotated)
print("=================")
cv2.waitKey(0)
cv2.destroyAllWindows()
  

13

: 